In [ ]:
# for the WSL conda environment
# 1. python kernel is named wslminiconda3 (Python 3.11.14)
# 2. the environment is named base and is in the directory /home/arhab/wslminiconda3
# 3. Do not use the python kernel named base

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import geopandas as gpd
# from ftplib import FTP
# import hvplot.xarray
import plotly.express as px

In [ ]:
scenario_option = 0
parameter_option = 5

multiple_scenario = 1

In [ ]:
presentation_type = "daily_mean" 
second_part=1
# monthly_mean , daily_mean , "daily_centre"

In [ ]:
if scenario_option == 0:
    scenario = "observational"
elif scenario_option ==1:
    scenario = "rcp26"
elif scenario_option ==2:
    scenario = "rcp45"
elif scenario_option ==3:
    scenario = "rcp85"


if parameter_option == 0:
    parameter = "tas"
elif parameter_option==1:
    parameter="pr"
elif parameter_option==2:
    parameter="huss"
elif parameter_option==3:
    parameter="rlds"
elif parameter_option==4:
    parameter="rsds"
elif parameter_option==5:
    parameter="sfcWind"

In [ ]:
# parameter_array = ["tas", "pr", "huss", "rlds",  "rsds", "sfcWind"]
# # parameter_array = ["sfcWind"]
# # parameter_array = [parameter]

# for parameter in parameter_array:
#     if scenario=="observational":
#         data = xr.open_mfdataset(rf"D:\thesis_data\new_cordex_data\{scenario}\{parameter}_EUR-11_ECMWF-ERAINT_{scenario}_r1i1p1_CNRM-ALADIN63_v1_day_*.nc")
#     else:
#         data = xr.open_mfdataset(rf"D:\thesis_data\new_cordex_data\{scenario}\{parameter}_EUR-11_CNRM-CERFACS-CNRM-CM5_{scenario}_r1i1p1_CNRM-ALADIN63_v2_day_*.nc")
    
#     data= data.where((data.lon>10.1).compute() & (data.lon<12.4).compute() & (data.lat>46.1).compute() & (data.lat<47.2).compute(), drop=True)

#     data = data[parameter]
#     # Assuming data2 is your xarray.Dataset
#     new_x_values = xr.DataArray([10.17544839, 10.33772344, 10.5 , 10.66227656, 10.82455161, 10.98682366, 11.14909118, 11.31135269, 11.47360667, 11.63585162, 11.79808604, 11.96030843, 12.12251727, 12.28471108],  dims=['x'])
#     new_y_values = xr.DataArray([46.12945789, 46.24168478, 46.3539238 , 46.46617454, 46.57843659, 46.69070955, 46.80299301, 46.91528657, 47.02758981, 47.13990233], dims=['y'])

#     # Create a new dataset with the modified 'y' values
#     data = data.assign_coords(x=new_x_values)
#     data = data.assign_coords(y=new_y_values)

#     output_file = rf'D:\thesis_data\new_cordex_data\{scenario}\cropped_data\{parameter}_{scenario}.nc'
#     data.to_netcdf(output_file, format='NETCDF4', engine='netcdf4')
#     print(f"Done {parameter}")


In [ ]:
if multiple_scenario ==1:
    scenario="rcp26"
    
input_file = rf'.\cropped_data\{scenario}\{parameter}_{scenario}.nc'
data = xr.open_mfdataset(input_file)

In [ ]:
if multiple_scenario == 1:
    # scenario="rcp45"
    input_file = rf'.\cropped_data\rcp45\{parameter}_rcp45.nc'
    data_rcp45 = xr.open_mfdataset(input_file)

In [ ]:
if multiple_scenario == 1:
    # scenario="rcp85"
    input_file = rf'.\cropped_data\rcp85\{parameter}_rcp85.nc'
    data_rcp85 = xr.open_mfdataset(input_file)


In [ ]:
scenario2 = "observational"
data_hist = xr.open_mfdataset(rf'.\cropped_data\{scenario2}\{parameter}_{scenario2}.nc')
data_hist_check = data_hist[parameter] * 0.5

if second_part==1:
    start_date = '2011-01-01'
    end_date = data_hist['time'].max()  # Get the maximum date in the 'time' coordinate
    data_hist = data_hist.sel(time=slice(start_date, end_date))

In [ ]:
data_rcp85

In [ ]:
# data2_updated[parameter].isel(y=5, x=7).plot()
# data[parameter].isel(y=5, x=7).plot()

In [ ]:
rcp_time = data["time"].values
hist_time = data_hist["time"].values

min_length = min(len(rcp_time), len(hist_time))

rcp_time=rcp_time[:min_length]
hist_time=hist_time[:min_length]


if presentation_type=="monthly_mean":
    rcp = data[parameter].mean(dim=('y', 'x')).resample(time="1M").mean(dim="time").values
    hist = data_hist[parameter].mean(dim=('y', 'x')).resample(time="1M").mean(dim="time").values

    rcp_time = data["time"].resample(time="1M").mean().values
    hist_time = data_hist["time"].resample(time="1M").mean().values
    hist_time=hist_time[:len(rcp_time)]

    if multiple_scenario==1:
        array_rcp45 = data_rcp45[parameter].mean(dim=('y', 'x')).resample(time="1M").mean(dim="time").values
        array_rcp85 = data_rcp85[parameter].mean(dim=('y', 'x')).resample(time="1M").mean(dim="time").values


if presentation_type=="daily_mean":
    rcp = data[parameter].mean(dim=('y', 'x')).values
    hist = data_hist[parameter].mean(dim=('y', 'x')).values

    if multiple_scenario==1:
        array_rcp45 = data_rcp45[parameter].mean(dim=('y', 'x')).values
        array_rcp85 = data_rcp85[parameter].mean(dim=('y', 'x')).values


if presentation_type=="daily_centre":
    rcp = data[parameter].isel(y=5, x=7).values
    hist = data_hist[parameter].isel(y=5, x=7).values

    if multiple_scenario==1:
        array_rcp45 = data_rcp45[parameter].isel(y=5, x=7).values
        array_rcp85 = data_rcp85[parameter].isel(y=5, x=7).values

In [ ]:
# time_df = pd.DataFrame(
#     {"rcp_time" : rcp_time,
#      "hist_time" : hist_time })
# time_df

In [ ]:
rcp = rcp.flatten()
hist = hist.flatten()

min_length = min(len(rcp), len(hist))

rcp = rcp[:min_length]
hist = hist[:min_length]

if multiple_scenario==1:
    array_rcp45 = array_rcp45[:min_length]
    array_rcp85 = array_rcp85[:min_length]

print(rcp.shape)
print(hist.shape)

In [ ]:
dict_temp = {
    "hist_time" : hist_time,
    "rcp_time" : rcp_time,
    "hist": hist,
    "rcp": rcp}

if multiple_scenario==1:
    dict_temp = {
    "hist_time" : hist_time,
    "rcp_time" : rcp_time,
    "hist": hist,
    "rcp26": rcp,
    "rcp45": array_rcp45,
    "rcp85": array_rcp85
    }

# Create the DataFrame
df = pd.DataFrame(dict_temp)

In [ ]:
# import plotly.express as px
# fig = px.line(df, x= "rcp_time", y=["hist", "rcp26", "rcp45", "rcp85"])
# fig.update_layout(title=f'{parameter} | Total points: {len(df)} | {presentation_type} | {scenario} vs {scenario2}')
# fig

In [ ]:
# fig = px.scatter(df, x= "rcp_time", y=["rcp", "hist"])
# fig.update_layout(title=f'{parameter} | Total points: {len(df)} | {presentation_type} | {scenario} vs {scenario2}')
# fig

In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd

# small_size=14
# big_size=16

# # Create figure and axes with a specific size
# fig, ax1 = plt.subplots(figsize=(30, 6))

# # Plot data on the first axis
# line1, = ax1.plot(df["rcp_time"], df[f'{scenario}'], 'b-', label=f'{scenario}')
# ax1.set_xlabel(f'Date {scenario}', color="b", fontsize=small_size)  # Adjust fontsize here
# ax1.set_ylabel(f'{parameter}', fontsize=big_size)  # Adjust fontsize here
# ax1.tick_params(axis='x', colors='b', labelsize=small_size)  # Adjust fontsize here for x-axis ticks
# ax1.tick_params(axis='y', labelsize=small_size)  # Adjust fontsize here for x-axis ticks

# # Create a second x-axis
# ax2 = ax1.twiny()

# # Plot data on the second axis
# line2, = ax2.plot(df["hist_time"], df["hist"], 'r-', label=f'{scenario2}')
# ax2.set_xlabel(f'Date {scenario2}', color="r", fontsize=small_size)  # Adjust fontsize here
# ax2.tick_params(axis='x', colors='r', labelsize=small_size)  # Adjust fontsize here for x-axis ticks

# if multiple_scenario==1:
#     ax3 = ax1.twiny()

#     # Plot data on the second axis
#     line3, = ax3.plot(df["rcp_time"], df["rcp45"], 'yellow', label='rcp45')
#     ax3.set_xlabel(f'Date rcp45', color="yellow", fontsize=small_size)  # Adjust fontsize here
#     ax3.tick_params(axis='x', colors='yellow', labelsize=small_size)  # Adjust fontsize here for x-axis ticks

#     ax4 = ax1.twiny()

#     # Plot data on the second axis
#     line4, = ax4.plot(df["rcp_time"], df["rcp85"], 'green', label='rcp85')
#     ax4.set_xlabel(f'Date rcp85', color="green", fontsize=small_size)  # Adjust fontsize here
#     ax4.tick_params(axis='x', colors='green', labelsize=small_size)  # Adjust fontsize here for x-axis ticks


# # Plot data on the second axis
# line2, = ax2.plot(df["hist_time"], df["hist"], 'r-', label=f'{scenario2}')
# ax2.set_xlabel(f'Date {scenario2}', color="r", fontsize=small_size)  # Adjust fontsize here
# ax2.tick_params(axis='x', colors='r', labelsize=small_size)  # Adjust fontsize here for x-axis ticks



# # Adding legends for both plots at upper right
# ax1.legend([line1], [f'{scenario}'], loc='upper right', bbox_to_anchor=(1, 1), fancybox=True, shadow=True, ncol=1, fontsize=small_size)  # Adjust fontsize here
# ax2.legend([line2], [f'{scenario2}'], loc='upper right', bbox_to_anchor=(1, 0.92), fancybox=True, shadow=True, ncol=1, fontsize=small_size)  # Adjust fontsize here

# plt.tight_layout()
# plt.title(f'{parameter} | Total points: {len(df)} | {presentation_type} | {scenario} vs {scenario2}', fontsize=big_size)
# plt.show()


In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

# Creating subplots
fig = go.Figure()

# Adding rcp26, rcp45, rcp85 data to the subplot
for col in ['rcp26', 'rcp45', 'rcp85']:
    fig.add_trace(go.Scatter(x=df['rcp_time'], y=df[col], mode='lines', name=col))

# Adding hist data to the subplot with a separate x-axis
fig.add_trace(go.Scatter(x=df['hist_time'], y=df['hist'], mode='lines', name='hist', xaxis='x2'))

# Define layout for subplot with two x-axes
fig.update_layout(
    xaxis=dict(title='rcp_time'),
    xaxis2=dict(title='hist_time', overlaying='x', side='top'),
    yaxis=dict(title=f'{parameter}'),
    title='Title'
)
fig.update_layout(title=f'{parameter} | Total points: {len(df)} | {presentation_type}')
fig.show()

pio.write_image(fig, rf'D:\thesis_data\notebooks\images\all_four_{parameter}_{presentation_type}.png', width=1800, height=600)

